In [1]:
from src.Autoencoder import Autoencoder
from src.Dataloader import Dataloader 
import torch
model = Autoencoder(4)
dataloader = Dataloader("Datasets/Dataset003/Train",24, 224, 224, True)

n_images = 4
images = dataloader.load_images(dataloader.path, n_images)
# print('Loaded images shape:', np.array(images).shape)
images = dataloader.greyscale_images(images)
# print('Greyscale images shape:', np.array(images).shape)
images = dataloader.layer_images(images, n_images)
print ('Layered images shape:', images.shape)
images = torch.tensor(images, dtype=torch.float32)




Layered images shape: (500, 4, 224, 224)


In [14]:
from torch.utils.data import random_split

# Define the loss function and optimizer

model.cuda()# Move the model to the GPU
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 0.001)

BS = 16
# Split the dataset into training and testing datasets
train_size = int(0.8 * len(images))
test_size = len(images) - train_size
train_dataset, test_dataset = random_split(images, [train_size, test_size])

# Create DataLoaders for training and testing datasets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BS, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BS, shuffle=True)

print('Train dataset size:', len(train_dataset))

print(type(train_loader))
print(train_loader.dataset[0].shape)

Train dataset size: 400
<class 'torch.utils.data.dataloader.DataLoader'>
torch.Size([4, 224, 224])


In [21]:
from tqdm import tqdm
import matplotlib.pyplot as plt
# Define a list to store training loss and validation loss
Loss = []
Validation_Loss = []


num_epochs = 100
for epoch in tqdm(range(num_epochs)):
    model.train()  # Set model to training mode
    # for img, _ in train_loader:
    for idx, img in enumerate(train_loader):
        # print('img shape:', img.shape)
        img = img.cuda()
        
        output = model(img)
        loss = criterion(output, img)

        optimizer.zero_grad() #clears the gradients of all optimized tensors.  This step is necessary because gradients are accumulated by default in PyTorch, and we want to compute fresh gradients for the current batch of data.
        loss.backward() # This line computes the gradients of the loss function with respect to the model parameters. These gradients are used to update the model parameters during optimization.
        optimizer.step() # This line updates the model parameters using the computed gradients. 
    Loss.append(loss.item())
       

    # Calculate validation loss
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_loss_sum = 0.0
        num_batches = 0
        for idx, img in enumerate(test_loader):
            img = img.cuda()
            output = model(img)
            val_loss = criterion(output, img)
            val_loss_sum += val_loss.item()
            num_batches += 1
        val_loss_avg = val_loss_sum / num_batches
        Validation_Loss.append(val_loss_avg)
    
    if epoch % 5 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}, Validation Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item(), val_loss_avg))

plt.plot(Loss, label='Training Loss')
plt.plot(Validation_Loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

  1%|          | 1/100 [00:01<02:03,  1.25s/it]

Epoch [1/100], Loss: 8550.3535, Validation Loss: 8507.1212


  6%|▌         | 6/100 [00:05<01:23,  1.13it/s]

Epoch [6/100], Loss: 8445.4629, Validation Loss: 8482.7157


  8%|▊         | 8/100 [00:07<01:28,  1.05it/s]


KeyboardInterrupt: 

In [ ]:
for idx, img in enumerate(train_loader):
    img = img.cuda()
    output = model(img)
    loss = criterion(output, img)
    print('img shape:', img.shape)
    print('output shape:', output.shape)
    print('loss shape:', loss.shape)
    break

0
img shape: torch.Size([16, 4, 224, 224])
output shape: torch.Size([16, 4, 224, 224])
loss shape: torch.Size([])
